In [3]:
# default_exp engineering.nbdev

%reload_ext autoreload
%autoreload 2

# tensorboard
TensorFlow中强大的可视化工具
## 使用流程
python脚本记录可视化数据

## 硬盘: event file
![](img/tfboard00.png)
## 打开web终端: tensorboard

命令行输入: 

In [14]:
!tensorboard --logdir ../../'Google 云端硬盘'/'Colab Notebooks'/  # run所在的文件夹

^C
Traceback (most recent call last):
  File "/Users/luoyonggui/anaconda3/bin/tensorboard", line 6, in <module>
    from tensorboard.main import run_main
  File "/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/tensorboard/main.py", line 43, in <module>
    from tensorboard import default
  File "/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/tensorboard/default.py", line 40, in <module>
    from tensorboard.plugins.beholder import beholder_plugin_loader
  File "/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/tensorboard/plugins/beholder/__init__.py", line 18, in <module>
    import tensorflow
  File "/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/tensorflow/__init__.py", line 48, in <module>
    from ._api.v2 import compat
  File "/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/tensorflow/_api/v2/compat/__init__.py", line 39, in <module>
    from . import v1
  File "/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/tensorflow/_api/v2/comp

### 常见报错
tensorboard ValueError: Duplicate plugins for name projector

https://blog.csdn.net/Avada_533/article/details/102223823

## install

In [10]:
!pip install tensorboard

     |████████████████████████████████| 6.8MB 484kB/s eta 0:00:01
     |████████████████████████████████| 788kB 268kB/s eta 0:00:01
ERROR: tensorflow 2.2.0 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.5.1 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 2.3.0 which is incompatible.
ERROR: snorkel 0.9.5 has requirement scikit-learn<0.22.0,>=0.20.2, but you'll have scikit-learn 0.23.1 which is incompatible.
ERROR: snorkel 0.9.5 has requirement tensorboard<2.0.0,>=1.14.0, but you'll have tensorboard 2.3.0 which is incompatible.


In [4]:
!pip freeze | grep grpcio

grpcio==1.20.0


In [9]:
!pip freeze | grep tensor

tensorflow==2.2.0
tensorflow-datasets==1.3.0
tensorflow-estimator==2.2.0
tensorflow-metadata==0.15.0
tensorlayer==1.11.1


# api

## SummaryWriter
功能:提供创建event file的高级接口  
主要属性:
* log_dir:event file输出文件夹 
* comment:不指定log_dir时，文件夹后缀
* filename_suffix: event file文件名后缀




### add_scalar()
功能:记录标量

    * tag:图像的标签名，图的唯一标识 
    * scalar_value:要记录的标量
    * global_step:x轴
    
    局限: 一个图只能记录一条曲线的变化。
### add_scalars()
    * main_tag:该图的标签
    * tag_scalar_dict:key是变量的tag，value是变量的值
    
    一个图记录多条曲线的变化
### add_histogram() 功能:统计直方图与多分位数折线图
    * tag:图像的标签名，图的唯一标识 
    * values:要统计的参数
    * global_step:y轴
    * bins:取直方图的bins

### 示例

In [ ]:
import numpy as np
from torch.utils.tensorboard import SummaryWriter


writer = SummaryWriter(comment='test_tensorboard')

for x in range(100):

    writer.add_scalar('y=2x', x * 2, x)
    writer.add_scalar('y=pow(2, x)',  2 ** x, x)
    
    writer.add_scalars('data/scalar_group', {"xsinx": x * np.sin(x),
                                             "xcosx": x * np.cos(x),
                                             "arctanx": np.arctan(x)}, x)
writer.close()

In [ ]:

    writer = SummaryWriter(comment='test_comment', filename_suffix="test_suffix")

    for x in range(2):

        np.random.seed(x)

        data_union = np.arange(100)
        data_normal = np.random.normal(size=1000)

        writer.add_histogram('distribution union', data_union, x)
        writer.add_histogram('distribution normal', data_normal, x)

        plt.subplot(121).hist(data_union, label="union")
        plt.subplot(122).hist(data_normal, label="normal")
        plt.legend()
        plt.show()

    writer.close()

In [ ]:
# 构建 SummaryWriter
writer = SummaryWriter(comment='test_your_comment', filename_suffix="_test_your_filename_suffix")

for epoch in range(MAX_EPOCH):

    loss_mean = 0.
    correct = 0.
    total = 0.

    net.train()
    for i, data in enumerate(train_loader):

        iter_count += 1

        # forward
        inputs, labels = data
        outputs = net(inputs)

        # backward
        optimizer.zero_grad()
        loss = criterion(outputs, labels)
        loss.backward()

        # update weights
        optimizer.step()

        # 统计分类情况
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).squeeze().sum().numpy()

        # 打印训练信息
        loss_mean += loss.item()
        train_curve.append(loss.item())
        if (i+1) % log_interval == 0:
            loss_mean = loss_mean / log_interval
            print("Training:Epoch[{:0>3}/{:0>3}] Iteration[{:0>3}/{:0>3}] Loss: {:.4f} Acc:{:.2%}".format(
                epoch, MAX_EPOCH, i+1, len(train_loader), loss_mean, correct / total))
            loss_mean = 0.

        # 记录数据，保存于event file
        writer.add_scalars("Loss", {"Train": loss.item()}, iter_count)
        writer.add_scalars("Accuracy", {"Train": correct / total}, iter_count)

    # 每个epoch，记录梯度，权值
    for name, param in net.named_parameters():
        writer.add_histogram(name + '_grad', param.grad, epoch)
        writer.add_histogram(name + '_data', param, epoch)

    scheduler.step()  # 更新学习率

    # validate the model
    if (epoch+1) % val_interval == 0:

        correct_val = 0.
        total_val = 0.
        loss_val = 0.
        net.eval()
        with torch.no_grad():
            for j, data in enumerate(valid_loader):
                inputs, labels = data
                outputs = net(inputs)
                loss = criterion(outputs, labels)

                _, predicted = torch.max(outputs.data, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).squeeze().sum().numpy()

                loss_val += loss.item()

            valid_curve.append(loss.item())
            print("Valid:\t Epoch[{:0>3}/{:0>3}] Iteration[{:0>3}/{:0>3}] Loss: {:.4f} Acc:{:.2%}".format(
                epoch, MAX_EPOCH, j+1, len(valid_loader), loss_val, correct / total))

            # 记录数据，保存于event file
            writer.add_scalars("Loss", {"Valid": np.mean(valid_curve)}, iter_count)
            writer.add_scalars("Accuracy", {"Valid": correct / total}, iter_count)

### add_image()
功能:记录图像

    • tag:图像的标签名，图的唯一标识 
    • img_tensor:图像数据，注意尺度 （当tensorboard检测到数据都位于0-1之间时，就会乘以255；否则不处理
    • global_step:x轴
    • dataformats:数据形式，CHW(默认)，HWC，HW (C: channel, H: height, W: width


#### torchvision.utils.make_grid 

功能:制作网格图像

    • tensor:图像数据, `B*C*H*W`形式
    • nrow:行数(列数自动计算)
    • padding:图像间距(像素单位)
    • normalize:是否将像素值标准化
    • range:标准化范围
    • scale_each:是否单张图维度标准化 
    • pad_value:padding的像素值


### add_graph() 
功能:可视化模型计算图

    • model:模型，必须是 nn.Module
    • input_to_model:输出给模型的数据 
    • verbose:是否打印计算图结构信息




### add_image示例

In [ ]:
    writer = SummaryWriter(comment='test_your_comment', filename_suffix="_test_your_filename_suffix")

    # img 1     random
    fake_img = torch.randn(3, 512, 512)
    writer.add_image("fake_img", fake_img, 1)
    time.sleep(1)

    # img 2     ones
    fake_img = torch.ones(3, 512, 512)
    time.sleep(1)
    writer.add_image("fake_img", fake_img, 2)

    # img 3     1.1
    fake_img = torch.ones(3, 512, 512) * 1.1
    time.sleep(1)
    writer.add_image("fake_img", fake_img, 3)

    # img 4     HW
    fake_img = torch.rand(512, 512)
    writer.add_image("fake_img", fake_img, 4, dataformats="HW")

    # img 5     HWC
    fake_img = torch.rand(512, 512, 3)
    writer.add_image("fake_img", fake_img, 5, dataformats="HWC")

    writer.close()

In [ ]:
    writer = SummaryWriter(comment='test_your_comment', filename_suffix="_test_your_filename_suffix")

    split_dir = os.path.join("..", "..", "data", "rmb_split")
    train_dir = os.path.join(split_dir, "train")
    # train_dir = "path to your training data"

    transform_compose = transforms.Compose([transforms.Resize((32, 64)), transforms.ToTensor()])
    train_data = RMBDataset(data_dir=train_dir, transform=transform_compose)
    train_loader = DataLoader(dataset=train_data, batch_size=16, shuffle=True)
    data_batch, label_batch = next(iter(train_loader))

    img_grid = vutils.make_grid(data_batch, nrow=4, normalize=True, scale_each=True)
    # img_grid = vutils.make_grid(data_batch, nrow=4, normalize=False, scale_each=False)
    writer.add_image("input img", img_grid, 0)

    writer.close()

In [ ]:
    writer = SummaryWriter(comment='test_your_comment', filename_suffix="_test_your_filename_suffix")

    # 模型
    fake_img = torch.randn(1, 3, 32, 32)

    lenet = LeNet(classes=2)

    writer.add_graph(lenet, fake_img)

    writer.close()


In [ ]:
from nbdev.export import notebook2script
notebook2script('.ipynb')